In [1]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10000'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10000'

In [2]:
import ee
import geemap
import math
from bqplot import pyplot as plt
from bqplot import Bars

In [3]:
def powerToDb(img):
    return ee.Image(10).multiply(img.log10())

def dbToPower(img):
    return ee.Image(10).pow(img.divide(10))

In [4]:
def gammaMap(img):
    ksize = 7
    enl = 5
    bandNames = img.bandNames()
    

    # Convert image from dB to natural values
    nat_img = dbToPower(img)

    # Square kernel, ksize should be odd (typically 3, 5 or 7)
    weights = ee.List.repeat(ee.List.repeat(1,ksize),ksize)

    # ~~(ksize/2) does integer division in JavaScript
    kernel = ee.Kernel.fixed(ksize,ksize, weights, int(3.5), int(3.5), False)

    # Get mean and variance
    mean = nat_img.reduceNeighborhood(ee.Reducer.mean(), kernel)
    variance = nat_img.reduceNeighborhood(ee.Reducer.variance(), kernel)

    # "Pure speckle" threshold
    ci = variance.sqrt().divide(mean);  # square root of inverse of enl

    # If ci <= cu, the kernel lies in a "pure speckle" area -> return simple mean
    cu = 1.0/math.sqrt(enl)

    # If cu < ci < cmax the kernel lies in the low textured speckle area -> return the filtered value
    cmax = math.sqrt(2.0) * cu

    alpha = ee.Image(1.0 + cu*cu).divide(ci.multiply(ci).subtract(cu*cu))
    b = alpha.subtract(enl + 1.0)
    d = mean.multiply(mean).multiply(b).multiply(b).add(alpha.multiply(mean).multiply(nat_img).multiply(4.0*enl))
    f = b.multiply(mean).add(d.sqrt()).divide(alpha.multiply(2.0))

    #问题在这里
    caster = ee.Dictionary.fromLists(bandNames,ee.List.repeat('float',3))
    img1 = powerToDb(mean.updateMask(ci.lte(cu))).rename(bandNames).cast(caster)
    img2 = powerToDb(f.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))).rename(bandNames).cast(caster)
    img3 = img.updateMask(ci.gte(cmax)).rename(bandNames).cast(caster)

    # If ci > cmax do not filter at all (i.e. we don't do anything, other then masking)
    result = ee.ImageCollection([img1,img2,img3]).reduce(ee.Reducer.firstNonNull()).rename(bandNames)
   

    # Compose a 3 band image with the mean filtered "pure speckle", the "low textured" filtered and the unfiltered portions
    return result

In [5]:
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)

    indices = ee.List.sequence(1, size)

  # Compute between sum of squares, where each mean partitions the data.

    def func_xxx(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = aMeans.multiply(aCounts) \
            .reduce(ee.Reducer.sum(), [0]).get([0]) \
            .divide(aCount)
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
              bCount.multiply(bMean.subtract(mean).pow(2)))

    bss = indices.map(func_xxx)

    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

In [6]:
Map = geemap.Map()
Map.setCenter(116.3518, 29.0403,9)
Map

Map(center=[29.0403, 116.3518], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [7]:
roi= ee.Geometry.Rectangle(115.7701, 29.7696, 116.8961, 28.3553) #鄱阳湖
poyang = ee.FeatureCollection('projects/validation-324408/assets/poyang')

In [8]:
vis = {'bands':['VV'],        
       'min':-25, 'max':0
}

# search NAIP imagery that has RGBN bands
collection = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(roi) \
    .filterDate('2020-07-15', '2020-07-30') \
    .filter(ee.Filter.listContains("system:band_names", "VV"))
image = collection.mean()
image1 = collection.first().clip(poyang)
# dem = ee.Image("MERIT/DEM/v1_0_3")
# dem = dem.lt(30)
Map.addLayer(image,vis,"sentinel-1")

In [9]:
filtered_SAR = gammaMap(image)
biocut = filtered_SAR.select('VV').lt(-16)
canny = ee.Algorithms.CannyEdgeDetector(biocut,1,1)
canny = canny.clip(roi).unmask()
connected  = canny.updateMask(canny).lt(0.05).connectedPixelCount(500, True)
edges = connected.gte(5)
edges = edges.updateMask(edges)
edgeBuffer = edges.focal_max(5, 'square', 'meters')
histogram_image = filtered_SAR.select('VV').updateMask(edgeBuffer)
histogram_ =  histogram_image.select('VV').reduceRegion(**{
  'reducer': ee.Reducer.histogram(255, 0.2),
  'geometry': roi,
  'scale': 10,
  'bestEffort': True
})
hist_dict = histogram_.getInfo()
threshold1 = otsu(histogram_.get('VV'))
waterImgPre = ee.Image(ee.Algorithms.If(False,filtered_SAR.gt(threshold1),filtered_SAR.lt(threshold1))).selfMask().clip(poyang)
waterImg = waterImgPre.select('VV')

Map.addLayer(waterImg, {'palette': 'blue'}, 'WaterImg')

In [10]:
x = hist_dict['VV']['bucketMeans']
y = hist_dict['VV']['histogram']
plt.bar(x, y)
plt.show()
# plt.show()

In [11]:
print('threshold', threshold1.getInfo())

threshold -15.699541989598117
